In [67]:
import os
import sys
import pandas as pd
import re
import numpy as np
from scipy import stats
import statsmodels.stats.multitest

project_dir = 'D:/Penn/proj/als_ptms/'

In [69]:
##
## Read in data
##

df = pd.read_csv(os.path.join(project_dir, "./results/spectral_counts.csv"))
df.head()

batchlist_df = pd.read_csv(os.path.join(project_dir, "./data/batchlist.txt"), header=None, sep="\t", engine="python")
batchlist_df.columns = ['FileName', 'SampleGroup']
batchlist_df['FileName'] = batchlist_df['FileName'].map(lambda x: re.search("TN_CSF_062617_[0-9]{2}", x)[0])

healthy_files = list(batchlist_df['FileName'].loc[batchlist_df['SampleGroup'] == 'H'])
als_files = list(batchlist_df['FileName'].loc[batchlist_df['SampleGroup'] == 'A'])


##
## Calculate statistics
##

# function to calculate t-test pvalue given a row with ALS and Healthy file values
def get_ttest_pval(row):    
    tstat, pval = stats.ttest_ind(row[als_files].values.ravel(), row[healthy_files].values.ravel())
    return(pval)

# iterate over rows (features) to calculate p-value
df['pval'] = df.apply(lambda x: get_ttest_pval(x), axis=1)

# multiple hypothesis correction
df['adj_pval'] = statsmodels.stats.multitest.multipletests(pvals=df['pval'])[1]

# function to calculate fold change
def get_foldchange(row):
    eps = sys.float_info.epsilon  # small constant to prevent divide-by-zero in next line
    fc = (sum(row[als_files].values.ravel())+eps)/(sum(row[healthy_files].values.ravel())+eps)
    return(fc)

# iterate over rows (features) to calculate fold change
df['raw_foldchange'] = df.apply(lambda x: get_foldchange(x), axis=1)
df['log2foldchange'] = df['raw_foldchange']
df.head()


,Feature,TN_CSF_062617_03,TN_CSF_062617_04,TN_CSF_062617_06,TN_CSF_062617_07,TN_CSF_062617_08,TN_CSF_062617_10,TN_CSF_062617_11,TN_CSF_062617_15,TN_CSF_062617_17,...,TN_CSF_062617_50,TN_CSF_062617_51,TN_CSF_062617_52,TN_CSF_062617_53,TN_CSF_062617_54,TN_CSF_062617_57,TN_CSF_062617_59,pval,adj_pval,raw_foldchange
0,LRVLSGHLLGRPREALSTNEC[160]K_125,3,2,1,2,3,0,0,0,0,...,0,0,0,0,0,0,0,0.000439,0.792755,3.363636e+00
1,TTMDPNDVILATHASVDNLLHLSGLLER_34,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0.001580,0.996548,4.053240e+16
2,DNELLHDAEMENYAHLRAQGGEVMEYTTILRLR_207,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0.001973,0.999155,1.100000e+01
3,THTC[160]PPC[160]PAPELLGGPSVFLFPPKPK_15,2,0,0,2,0,0,0,0,0,...,1,0,0,0,0,0,0,0.002192,0.999614,4.428571e+00
4,AIAANEADAVTLDAGLVYDAYLAPNNLKPVVAEFYGSK_48,0,2,0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0.002879,0.999967,8.500000e+00


In [70]:
# write out the results for dissemination
df.to_csv(os.path.join(project_dir, './results/openmodsearch_als_ptms_statanalysis.csv'), index=False)